In [1]:
!pip -q install streamlit
!pip -q install pyngrok

     |████████████████████████████████| 8.2MB 12.8MB/s 
     |████████████████████████████████| 112kB 66.5MB/s 
     |████████████████████████████████| 4.2MB 57.1MB/s 
     |████████████████████████████████| 174kB 78.0MB/s 
     |████████████████████████████████| 81kB 8.6MB/s 
     |████████████████████████████████| 92kB 9.2MB/s 
     |████████████████████████████████| 122kB 72.1MB/s 
     |████████████████████████████████| 71kB 6.8MB/s 
ERROR: google-colab 1.0.0 has requirement ipykernel~=4.10, but you'll have ipykernel 5.5.5 which is incompatible.
     |████████████████████████████████| 747kB 16.4MB/s 


In [2]:
!pip -q install category_encoders 

     |████████████████████████████████| 81kB 4.9MB/s 


In [3]:
from pyngrok import ngrok
from sklearn.preprocessing import LabelEncoder
import pickle
import pandas as pd
import numpy as np
import category_encoders as ce

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
%%writefile app.py

import pandas as pd
import numpy as np
import pickle
import re
import streamlit as st
from pyngrok import ngrok
from sklearn.externals import joblib
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
from lightgbm import LGBMRegressor
import category_encoders as ce


uploaded_file = st.file_uploader("Choose a file")
if uploaded_file is not None:
  data_point = pd.read_csv(uploaded_file)
  with open('/content/drive/MyDrive/CaseStudy1/x_train.pkl', 'rb') as fp:
    x_train = pickle.load(fp)
  gbdt_model = pickle.load(open('/content/drive/MyDrive/CaseStudy1/gbdt_clf.pkl','rb'))
  #Replacing True or False with 0,1 values in onpromotion and replacing the NAN values with 0(False) in test data
  data_point['onpromotion'] = data_point['onpromotion'].fillna(0)
  data_point['onpromotion'] = data_point['onpromotion'].astype(int)
  #Encoding all the category variables
  # specify categorical columns
  cat_cols = ['city', 'state', 'type','family']
  le = ce.OrdinalEncoder(cols=cat_cols,
                       return_df=True,
                       handle_unknown='impute') #Handling Unseen labels in test data
  x_train = le.fit_transform(x_train)
  data_point = le.transform(data_point)
  unit_sales = gbdt_model.predict(data_point)
  st.write(unit_sales)

Writing app.py


In [5]:
!ngrok authtoken 1s9EWYEkYGU9J6chwq2Z0Hucw5R_2iBbA53YyhmegQY3b67nm

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [6]:
public_url = ngrok.connect(port='80')
print (public_url)
!streamlit run --server.port 80 app.py >/dev/null

NgrokTunnel: "http://76647c54033c.ngrok.io" -> "http://localhost:80"
2021-05-23 06:44:13.444 An update to the [server] config option section was detected. To have these changes be reflected, please restart streamlit.
/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/usr/local/lib/python3.7/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and wi

## ***Model Deployment***

##### Click [here](https://youtu.be/SYuYhiVm5W0)